In [1]:
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [146]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') # get data from web
len(df) # len of df, there are 3 tables

3

In [147]:
df_pc_toronto = df[0] # 1st df is our necesary df
df_pc_toronto.shape

(180, 3)

Remove rows with 'Borough' equal 'Not assigned'		

In [148]:
df_pc_toronto.drop(df_pc_toronto[df_pc_toronto['Borough'] == 'Not assigned'].index, axis=0, inplace = True)
df_pc_toronto['Borough'].value_counts()


North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [150]:
df_pc_toronto.reset_index(drop=True, inplace=True)

In [155]:
df_pc_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [156]:
df_pc_toronto.shape       

(103, 3)

In [157]:
df_pc_toronto[df_pc_toronto['Neighbourhood'] == 'Not assigned'] # There is no Not assigned cells

,Postal Code,Borough,Neighbourhood


In [158]:
print(df_pc_toronto.head())
print('Shape:', (df_pc_toronto.shape))

  Postal Code           Borough                                Neighbourhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
Shape: (103, 3)


In [159]:
df_pc_toronto.isna().count()

Postal Code      103
Borough          103
Neighbourhood    103
dtype: int64

In [160]:
df_pc = pd.read_csv('/home/cat/Downloads/Geospatial_Coordinates.csv')

In [161]:
df_pc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [162]:
df_pc_toronto['Latitude'] = "" # add empty columns for ll
df_pc_toronto['Longitude'] = ""

In [164]:
df_pc_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",,
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",,


Take post code from df_pc_toronto, seeking it in df_pc, write Latitude and Longitude 

In [165]:
for data in df_pc_toronto.index:
    pc = df_pc_toronto.loc[data, 'Postal Code']
    latitude = df_pc[df_pc['Postal Code']==pc].Latitude.values[0]
    longitude = df_pc[df_pc['Postal Code']==pc].Longitude.values[0]
    df_pc_toronto.loc[data, 'Latitude'] = latitude
    df_pc_toronto.loc[data, 'Longitude'] = longitude   

In [166]:
df_pc_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


In [210]:
df_pc_toronto.shape

(103, 5)

In [169]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_t = location.latitude
longitude_t = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_t, longitude_t))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [179]:

map_toronto = folium.Map(location=[latitude_t, longitude_t], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_pc_toronto['Latitude'], df_pc_toronto['Longitude'], df_pc_toronto['Borough'], df_pc_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [180]:
CLIENT_ID = 'CKOGO1DFIN5ZJSF1CVGE1QOQ5LRBVXW0YQILFLVPURQLLIPN' # your Foursquare ID
CLIENT_SECRET = 'YVWUPDHTKTDGLBV33XJBHKVELZ4X1A041U3YGWG5C5XZJ0BR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [181]:
neighborhood_latitude = df_pc_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_pc_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_pc_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [237]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 800 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CKOGO1DFIN5ZJSF1CVGE1QOQ5LRBVXW0YQILFLVPURQLLIPN&client_secret=YVWUPDHTKTDGLBV33XJBHKVELZ4X1A041U3YGWG5C5XZJ0BR&v=20180605&ll=43.7532586,-79.3296565&radius=800&limit=100'

In [238]:
results = requests.get(url).json()

In [239]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [240]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/cat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,DVP at York Mills,Road,43.758899,-79.334099
3,TTC Stop #09083,Bus Stop,43.759655,-79.332223
4,TTC Stop 9083,Bus Stop,43.759251,-79.334000


In [241]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [242]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [243]:
toronto_venues = getNearbyVenues(names=df_pc_toronto['Neighbourhood'],
                                   latitudes=df_pc_toronto['Latitude'],
                                   longitudes=df_pc_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [245]:
print(toronto_venues.shape)
toronto_venues.head()

(2153, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [246]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4


In [247]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


In [248]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [249]:
toronto_onehot.shape

(2153, 269)

In [250]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.029412,0.0,0.0,0.0,0.0,0.00
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.111111,0.000000,0.0,0.0,0.0,0.0,0.00


In [251]:
toronto_grouped.shape

(96, 269)

In [252]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

----Agincourt----
                        venue  freq
0                      Lounge  0.25
1                Skating Rink  0.25
2   Latin American Restaurant  0.25
3              Breakfast Spot  0.25
4                 Yoga Studio  0.00
5          Mexican Restaurant  0.00
6  Modern European Restaurant  0.00
7           Mobile Phone Shop  0.00
8          Miscellaneous Shop  0.00
9   Middle Eastern Restaurant  0.00


----Alderwood, Long Branch----
                  venue  freq
0           Pizza Place  0.25
1              Pharmacy  0.12
2           Coffee Shop  0.12
3        Sandwich Place  0.12
4                   Pub  0.12
5          Dance Studio  0.12
6                   Gym  0.12
7  Other Great Outdoors  0.00
8   Moroccan Restaurant  0.00
9        Medical Center  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.10
1          Coffee Shop  0.10
2          Bridal Shop  0.05
3       Ice Cream Shop  0.05
4     Sushi Restaura

                       venue  freq
0             Clothing Store  0.09
1                Coffee Shop  0.09
2                       Café  0.03
3             Cosmetics Shop  0.03
4        Japanese Restaurant  0.03
5            Bubble Tea Shop  0.03
6         Italian Restaurant  0.03
7                    Theater  0.02
8  Middle Eastern Restaurant  0.02
9           Ramen Restaurant  0.02


----Glencairn----
                      venue  freq
0               Pizza Place   0.2
1             Metro Station   0.2
2       Japanese Restaurant   0.2
3                       Pub   0.2
4                      Park   0.2
5         Martial Arts Dojo   0.0
6            Massage Studio   0.0
7            Medical Center   0.0
8  Mediterranean Restaurant   0.0
9               Men's Store   0.0


----Golden Mile, Clairlea, Oakridge----
                venue  freq
0              Bakery   0.2
1            Bus Line   0.2
2        Intersection   0.1
3       Metro Station   0.1
4      Ice Cream Shop   0.1
5         B

                        venue  freq
0                        Park  0.50
1                  Playground  0.25
2                       Trail  0.25
3                 Yoga Studio  0.00
4               Metro Station  0.00
5  Modern European Restaurant  0.00
6           Mobile Phone Shop  0.00
7          Miscellaneous Shop  0.00
8   Middle Eastern Restaurant  0.00
9          Mexican Restaurant  0.00


----Roselawn----
                             venue  freq
0                           Garden  0.33
1                      Music Venue  0.33
2                     Home Service  0.33
3                      Yoga Studio  0.00
4              Moroccan Restaurant  0.00
5  Molecular Gastronomy Restaurant  0.00
6       Modern European Restaurant  0.00
7                Mobile Phone Shop  0.00
8               Miscellaneous Shop  0.00
9        Middle Eastern Restaurant  0.00


----Rouge Hill, Port Union, Highland Creek----
                             venue  freq
0                              Bar   1.0
1  

In [253]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [269]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
print(columns)
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


['Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Ethiopian Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Dance Studio,Pharmacy,Pub,Coffee Shop,Airport Service,College Rec Center,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Bridal Shop,Deli / Bodega,Shopping Mall,Middle Eastern Restaurant,Sushi Restaurant,Ice Cream Shop,Frozen Yogurt Shop
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Women's Store,Thai Restaurant,Juice Bar,Fast Food Restaurant,Indian Restaurant,Butcher
...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",Ramen Restaurant,Sushi Restaurant,Restaurant,Pizza Place,Sandwich Place,Café,Coffee Shop,Lounge,Japanese Restaurant,Juice Bar
92,"Willowdale, Willowdale West",Pharmacy,Grocery Store,Pizza Place,Coffee Shop,Bank,Butcher,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
93,Woburn,Coffee Shop,Soccer Field,Korean Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
94,Woodbine Heights,Park,Video Store,Beer Store,Bus Stop,Dance Studio,Athletics & Sports,Curling Ice,Skating Rink,Pharmacy,Dessert Shop


In [271]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1,
       0, 1, 0, 1, 1, 1, 1, 0], dtype=int32)

In [272]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Ethiopian Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Dance Studio,Pharmacy,Pub,Coffee Shop,Airport Service,College Rec Center,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Bridal Shop,Deli / Bodega,Shopping Mall,Middle Eastern Restaurant,Sushi Restaurant,Ice Cream Shop,Frozen Yogurt Shop
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Women's Store,Thai Restaurant,Juice Bar,Fast Food Restaurant,Indian Restaurant,Butcher
...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",Ramen Restaurant,Sushi Restaurant,Restaurant,Pizza Place,Sandwich Place,Café,Coffee Shop,Lounge,Japanese Restaurant,Juice Bar
92,"Willowdale, Willowdale West",Pharmacy,Grocery Store,Pizza Place,Coffee Shop,Bank,Butcher,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
93,Woburn,Coffee Shop,Soccer Field,Korean Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
94,Woodbine Heights,Park,Video Store,Beer Store,Bus Stop,Dance Studio,Athletics & Sports,Curling Ice,Skating Rink,Pharmacy,Dessert Shop


In [282]:
neighborhoods_venues_sorted['Cluster Labels'].isna().sum()

0

In [283]:
df_pc_toronto


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069
99,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.6627,-79.3216
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,-79.4985


In [273]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_pc_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted)

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7533,-79.3297,1.0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Ethiopian Restaurant,Donut Shop
1,M4A,North York,Victoria Village,43.7259,-79.3156,1.0,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Dance Studio,Pharmacy,Pub,Coffee Shop,Airport Service,College Rec Center,Ethiopian Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,1.0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Bridal Shop,Deli / Bodega,Shopping Mall,Middle Eastern Restaurant,Sushi Restaurant,Ice Cream Shop,Frozen Yogurt Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648,1.0,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,1.0,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Women's Store,Thai Restaurant,Juice Bar,Fast Food Restaurant,Indian Restaurant,Butcher


In [288]:

toronto_merged.dropna(inplace=True) # there were NaN, I couldnt change color for next step couse NaN


In [289]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters